# This is the [original notebook](https://github.com/malariagen/PDNA_SP_markers/blob/9b73a05d2c573e66ea70b282adb7055a66e2022e/work/3_haplotype_plots/20221221_haplotype_plots.ipynb) Richard wanted us to update and use:



# Introduction

In [1]:
!date
!hostname
!pwd

Wed Dec 21 19:06:20 GMT 2022
node-10-1-4
/nfs/team112_data03/personal/rp7/src/github/malariagen/PDNA_SP_markers/work/3_haplotype_plots


In [2]:
%run ../setup_abox.ipynb

python 3.8.6 (default, Oct 13 2020, 20:37:26) 
[GCC 8.3.0]
numpy 1.21.4
scipy 1.4.1
pandas 1.2.3
pysam 0.15.4
petlx 1.0.3
vcf 0.6.8
h5py 2.10.0
zarr 2.4.0
scikit-allel 1.2.1


In [3]:
# Inputs
vcf_file_format = "/lustre/scratch124/gsu/legacy/pipelines/builds/pf_70_build/pf_70_internal_release/vcf/%s.pf7.vcf.gz" # Location of VCFs, allowing chrom to vary
samples_fn = '/lustre/scratch126/gsu/team112/pf7/ftp_20221021/Pf7_samples.txt'
ref_genome_fn = '/lustre/scratch124/gsu/legacy/pfalciparum/resources/Pfalciparum.genome.fasta'
gff_fn = '/lustre/scratch124/gsu/legacy/pfalciparum/resources/snpEff/data/Pfalciparum_GeneDB_Feb2020/Pfalciparum_replace_Pf3D7_MIT_v3_with_Pf_M76611.gff'


In [4]:
population_colours = collections.OrderedDict()
# population_colours['SA']     = "#4daf4a"
population_colours['AF-W']= "#e31a1c"
population_colours['AF-C'] = "#fd8d3c" 
population_colours['AF-NE'] = "#bb8129" 
population_colours['AF-E'] = "#fecc5c"
# population_colours['AS-S-E'] = "#dfc0eb" 
# population_colours['AS-S-FE'] = "#984ea3" 
# population_colours['AS-SE-W'] = "#9ecae1"
# population_colours['AS-SE-E'] = "#3182bd"
# population_colours['OC-NG'] = "#f781bf"

In [5]:
# locations with at least 25 samples (see 20210218_drm_frequency_over_time_pops_adminlevel1.ipynb) (borderline exceptions are marked on this notebook)
# I think we should be able to generate this with code, but for now this is fine
locations = [

#     ('SA', 'Colombia', 'Cauca'),

    ('AF-W', 'Gabon', 'Wouleu-Ntem'),
    ('AF-W', 'Cameroon', 'Sud-Ouest'),
    ('AF-W', 'Nigeria', 'Lagos'),
    ('AF-W', 'Benin', 'Littoral'),
    ('AF-W', 'Benin', 'Atlantique'),
    ('AF-W', 'Ghana', 'Volta'),
    ('AF-W', 'Ghana', 'Greater Accra'),
    ('AF-W', 'Ghana', 'Upper East'),
    ('AF-W', 'Ghana', 'Central'),
    ('AF-W', 'Ghana', 'Ashanti'),
    ('AF-W', 'Ghana', 'Brong Ahafo'),
    ('AF-W', 'Burkina Faso', 'Haut-Bassins'),
    ('AF-W', 'Mali', 'Segou'),
    ('AF-W', 'Mali', 'Sikasso'),
    ('AF-W', 'Mali', 'Koulikoro'),
    ('AF-W', 'Mali', 'Bamako'),
    ('AF-W', 'Mali', 'Kayes'),
    
    
    ('AF-W', "Côte d'Ivoire", 'Abidjan'),
    ('AF-W', 'Mauritania', 'Hodh el Gharbi'),
    ('AF-W', 'Mauritania', 'Hodh ech Chargui'), 
    ('AF-W', 'Guinea', 'Nzerekore'),
    ('AF-W', 'Guinea', 'Faranah'),
    ('AF-W', 'Senegal', 'Sedhiou'),
    ('AF-W', 'Senegal', 'Dakar'),
    ('AF-W', 'Gambia', 'Upper River'),
    ('AF-W', 'Gambia', 'North Bank'),
    ('AF-W', 'Gambia', 'Western'),
    
    ('AF-C', 'DRC', 'Kinshasa'),

    ('AF-NE', 'Kenya', 'Kisumu'),
    ('AF-NE', 'Sudan', 'Khartoum'),

    ('AF-E', 'Kenya', 'Kilifi'),
    ('AF-E', 'Mozambique', 'Gaza'), 
    ('AF-E', 'Tanzania', 'Lindi'),
    ('AF-E', 'Tanzania', 'Tanga'),
    ('AF-E', 'Tanzania', 'Morogoro'),
    ('AF-E', 'Tanzania', 'Kagera'),
    ('AF-E', 'Tanzania', 'Kigoma'),
    ('AF-E', 'Malawi', 'Zomba'),   
    ('AF-E', 'Malawi', 'Chikwawa'),
    
#     ('AS-S-E', 'India', 'West Bengal'),
#     ('AS-S-E', 'India', 'Odisha'),

#     ('AS-S-FE', 'Bangladesh', 'Chittagong'),
#     ('AS-S-FE', 'India', 'Tripura'),

#     ('AS-SE-W', 'Thailand', 'Tak'),
#     ('AS-SE-W', 'Myanmar', 'Tanintharyi'),
#     ('AS-SE-W', 'Myanmar', 'Shan'),
#     ('AS-SE-W', 'Myanmar', 'Kayin'),
#     ('AS-SE-W', 'Myanmar', 'Kachin'),
#     ('AS-SE-W', 'Myanmar', 'Bago'),
#     ('AS-SE-W', 'Myanmar', 'Mandalay'),
#     ('AS-SE-W', 'Myanmar', 'Sagaing'),

#     ('AS-SE-E', 'Vietnam', 'Khanh Hoa'),
#     ('AS-SE-E', 'Vietnam', 'Ninh Thuan'),
#     ('AS-SE-E', 'Vietnam', 'Gia Lai'),
#     ('AS-SE-E', 'Vietnam', 'Dak Lak'),
#     ('AS-SE-E', 'Vietnam', 'Quang Nam'),
#     ('AS-SE-E', 'Vietnam', 'Dak Nong'),
#     ('AS-SE-E', 'Vietnam', 'Quang Tri'),
#     ('AS-SE-E', 'Vietnam', 'Binh Phuoc'),
#     ('AS-SE-E', 'Cambodia', 'Ratanakiri'),
#     ('AS-SE-E', 'Cambodia', 'Stueng Traeng'),
#     ('AS-SE-E', 'Cambodia', 'Preah Vihear'),
#     ('AS-SE-E', 'Cambodia', 'Pursat'),
#     ('AS-SE-E', 'Cambodia', 'Battambang'),
#     ('AS-SE-E', 'Cambodia', 'Pailin'),
    
#     ('AS-SE-E', 'Laos', 'Sekong'),
#     ('AS-SE-E', 'Laos', 'Attapeu'),
#     ('AS-SE-E', 'Laos', 'Salavan'),
#     ('AS-SE-E', 'Laos', 'Champasak'),
#     ('AS-SE-E', 'Laos', 'Savannakhet'),
#     ('AS-SE-E', 'Thailand', 'Sisakhet'),

#     ('OC-NG', 'Papua New Guinea', 'Milne Bay'),
#     ('OC-NG', 'Papua New Guinea', 'Madang'),
#     ('OC-NG', 'Papua New Guinea', 'East Sepik'),
#     ('OC-NG', 'Indonesia', 'Papua')
]       


# Functions

In [6]:
def determine_cds(gene_id='PF3D7_0709000', gff_fn=gff_fn):
    df_gff = allel.gff3_to_dataframe(gff_fn, attributes=['ID', 'Name'])
    gene_name = df_gff.loc[df_gff['ID'] == gene_id, 'Name'].values[0]
    if (gene_name == '') or (gene_name == '.'):
        gene_name=gene_id
    chrom = df_gff.loc[df_gff['ID'] == gene_id, 'seqid'].values[0]
    strand = df_gff.loc[df_gff['ID'] == gene_id, 'strand'].values[0]
    df_cds = df_gff.loc[
        ( df_gff['ID'].str.startswith(gene_id) )
        & ( df_gff['type'] == 'CDS' )
    ]
    return(
        {
            'gene_name':gene_name,
            'chrom':chrom,
            'strand':strand,
            'starts':df_cds['start'].values,
            'ends':df_cds['end'].values,
        }
    )

determine_cds()

{'gene_name': 'CRT',
 'chrom': 'Pf3D7_07_v3',
 'strand': '+',
 'starts': array([403222, 403490, 403938, 404283, 404569, 404764, 404936, 405146,
        405334, 405539, 405825, 406017, 406241]),
 'ends': array([403312, 403758, 404110, 404415, 404640, 404839, 405018, 405196,
        405390, 405631, 405869, 406071, 406317])}

In [7]:
####################################
# call_haplotype function
####################################

# This function creates nucleotide and amino acid haplotypes of specific regions, for example:
# 1) A single amino acid such as CRT 76
# 2) A stretch of contiguous amino acids such as CRT 72-76
# 3) The full coding region of a gene such as CRT

# Note that in the case of 3), if the gene has multiple exons, the region of interest will be non-contiguous

# The function can be used to call a) a single sample, b) specific samples of interest or c) all samples in the VCF (the default).
# Note that calling all samples isn't much slower than calling a single sample. Calling all haplotypes for all Pf7 samples took ~40 minutes

# The function returns a tuple with three elements:
# 1) A pandas DataFrame with sample ID as index, and three columns aa_haplotype, nucleotide_haplotype and ns_changes
# 2) The (3D7) reference amino acid haplotype
# 3) The (3D7) reference nucleotide haplotype

# Note that the reference haplotypes are probably not required. They were largely intended for debugging/sanity checking purposes
# As such the key output is the first element (the DataFrame)

####################################
# Explanation of column aa_haplotype
####################################
# If the nucleotide haplotype is homozygous (i.e. there are no heterozygous variants), there will be a single amino acid haplotype
# If the nucleotide haplotype is heterozygous (i.e. there is at least one heterozygous variants), there will be two amino acid haplotypes, separated by a comma
# Note that in some cases, there could be a heterozygous nucleotide haplotype, both alleles of which give the same amino acid haplotype.
# In cases such as this, the amino acid haplotype will be repeated, separated by a comma 
# If there are multiple mutations that are heterozygous, and cannot be phased, an asterisk (*) will be appended to the aa_haplotype call.
# If any variant is missing, aa_haplotype will be '-'.
# If the resulting nucleotide haplotype has a length that is not a multiple of 3 (due to indels) the aa_haplotype will be "!" to represent a frameshift
# In my experience to date, such frameshifts have only be seen as heterozygote calls, i.e. "<haplotype>,!" or "!,<haplotype>", though would
# probably be much more common if setting `include_indels=False`

####################################
# Explanation of column nucleotide_haplotype
####################################
# If the nucleotide haplotype is homozygous (i.e. there are no heterozygous variants), there will be a single haplotype
# If the nucleotide haplotype is heterozygous (i.e. there is at least one heterozygous variants), there will be two haplotypes, separated by a comma
# Nucleotides which match the reference sequence are shown in lower case, whereas nucleotides that differ (SNPs) are shown in upper case
# If any variant is missing, nucleotide_haplotype will be '-'.

####################################
# Explanation of column ns_changes
####################################
# All non-synonymous mutations will be shown in the form "<REF AA><AA number><ALT AA>", e.g. "K76T" or "C580Y"
# Homozygous mutations are shown in upper case and heterozygous in lower case.
# If the code finds two distinct amino acid haplotypes, these will be separated by a comma.
# If multiple NS mutations are found within an amino acid haplotype, these will be separated by a forward slash (/).
# If there are multiple mutations that are heterozygous, and cannot be phased, an asterisk (*) will be appended to the ns_changes call.
# Note that this still holds
# If there are no NS mutations, ns_changes will be NaN, with the exception of 2 or more unphased synonymous variants, in which case ns_changes will be '*'
# This use of '*' to represent multiple unphased synonymous mutations is perhaps undesirable and we might want to review this in the future
# Missing variants will be ignored, but any NS mutation elsewhere will still be included. Note the consequence of this is that it is
# possible that aa_haplotype will be '-' but ns_changes will have one or more NS mutations reported.
# If the resulting nucleotide haplotype has a length that is not a multiple of 3 (due to indels) ns_changes will be "!" to represent a frameshift
# If any mutations result in a premature stop codon, ns_changes will also be "!"

####################################
# Outline of the code
####################################
# This function is complex and difficult to follow (I had trouble coming back to it a few months after initially
# writing it). In essence this determines the reference nucleotide sequence of the region (ref_sequence), and then
# for each sample applies the mutations for the first and second aa_haplotype calls (remembering there are two calls as
# we are calling as diploid with GATK). This takes account of complications such as phased het calls and indels changing
# the length of the sequence (though indels are not included by default). The reference and two alternative nucleotide
# sequences are then converted to amino acid sequences, and mutations detected.

####################################
# Known issues
####################################
# Note that if using the default option of `include_indels=False` we shouldn't really see any frameshifts (represented by "!" in aa_haplotype),
# however, these are occasionally seen in the following two circumstances:
# 1) By default we allow the 1bp insertion and deletion at Pf3D7_07_v3:403618 and Pf3D7_07_v3:403622
#   (due to default option `positions_to_allow=[403618, 403622]`). These are usually seen together, so result in in-frame changes
#   However, in a few cases only one of the indels is called resulting in a frameshift
# 2) In some cases a SNP is represented by GATK as REF and ALT alleles longer than 1 but only differing in the first position.
#    An example of this in Pf7 is `Pf3D7_08_v3	549683	.	TGG	CGG,*`. There is a bug in the code that I haven't had chance to look into
#    which results in samples with this SNP being represented as a frameshift
#
# It is possible (and in fact quite common) to have a missing aa_haplotype (represented by "-"), but to have one or more mutations in ns_changes.
# This was by design and intended specifically for the gene Kelch13 where if we see any non-synonymous mutation between AA 349 and 726
# we can assign a resistant phenotype, even if there is missingness in other positions. In a sense aa_haplotype is more conservative than
# ns_changes. We might want to review this decision in future
#
# We have used '*' to represent multiple unphased synonymous mutations in ns_changes.
# This is perhaps undesirable and we might want to review this in the future.
#
# The current function requires a VCF file. This would probably be much quicker to run if using a zarr representation of the VCF

def call_haplotype(
#     target_name='crt',
    chrom='Pf3D7_07_v3',
    starts=[403222, 403490, 403938, 404283, 404569, 404764, 404936, 405146, 405334, 405539, 405825, 406017, 406241], # Start positions of each exon
    ends=  [403312, 403758, 404110, 404415, 404640, 404839, 405018, 405196, 405390, 405631, 405869, 406071, 406317], # End positions of each exon
    strand='+', # Either '+' or '-'
    vcf_file_format=vcf_file_format, # Location of VCFs, allowing chrom to vary
    samples=None, # The default (None) will call haplotypes for all samples within the VCF
    include_indels=False, # If set to True, this will include indels, and hence potentially output sequences of different lengths
    positions_with_missingness=[403620, 403621], # If there are missing genotypes at these positions, these will not be treated as missing. This is important in CRT as many samples have missing genotypes at Pf3D7_07_v3:403620-403621 due to way haplotypes are treated by HaplotypeCaller
    positions_to_ignore=[], # These positions will be skipped
    positions_to_allow=[403618, 403622], # These allows certain indels to be included even if include_indels is False. Important as common CVIET haplotype in CRT 72-76 is treated by HaplotypeCaller as two 1bp indels, rather than 3 SNPs
    ref_fasta_fn=ref_genome_fn,
    vcf_fn=None, # If supplied, this will overrule what is provided in vcf_file_format
    verbose=False,
    show_genotypes=False, # If true, these will give very verbose output
    use_filter=True, # If true, will only use PASS variants
    first_aa=1, # Important to use if, for example, only creating haplotypes for one exon
    to_stop=True # This argument is used by Bio translate function. If True, sequences will be truncated at premature stop codons, resulting in shorter AA sequences
):
    
    from pyfasta import Fasta
    from Bio.Seq import Seq
    
    if not isinstance(starts, list) and not(isinstance(starts, np.ndarray)):
        if isinstance(starts, int):
            starts = [starts]
        else:
            sys.exit("starts must be a list of integers (or a single integer)")
    if not isinstance(ends, list) and not(isinstance(ends, np.ndarray)):
        if isinstance(ends, int):
            ends = [ends]
        else:
            sys.exit("ends must be a list of integers (or a single integer)")
    
    if len(starts) != len(ends):
        sys.exit("starts and ends must be the same length")
        
    exon_offsets = [0] + list(np.cumsum(np.array(ends)-np.array(starts)+1))[:-1]
    
    if vcf_fn is None:
        vcf_fn = vcf_file_format % chrom
    
    ref_sequences = []
    for i in np.arange(len(starts)):
        start = starts[i]
        end = ends[i]
        ref_sequences.append(Seq(Fasta(ref_fasta_fn)[chrom][starts[i]-1:ends[i]]))
    ref_sequence = Seq("")
    for s in ref_sequences:
        ref_sequence += s
    ref_nucleotide_haplotype = str(ref_sequence)
    if strand == '+':
        ref_aa_haplotype = str(ref_sequence.translate(to_stop=to_stop))
    else:
        ref_aa_haplotype = str(ref_sequence.reverse_complement().translate(to_stop=to_stop))

    vcf_reader = vcf.Reader(filename=vcf_fn)
    if samples is None:
        samples = vcf_reader.samples
    sample_sequences = collections.OrderedDict()
    sample_offsets = collections.OrderedDict()
    PID = collections.OrderedDict()
    non_phased_het_seen = collections.OrderedDict()
    sample_unphaseable = collections.OrderedDict()
    aa_haplotype = collections.OrderedDict()
    nucleotide_haplotype = collections.OrderedDict()
    ns_changes = collections.OrderedDict()
    num_missing = collections.OrderedDict()
    num_called = collections.OrderedDict()
    swap_phasing = collections.OrderedDict()
    first_PGT = collections.OrderedDict()
    for sample in samples:
        sample_sequences[sample] = [ref_sequence.tomutable(), ref_sequence.tomutable()]
        sample_offsets[sample] = [0, 0]
        PID[sample] = ''
        non_phased_het_seen[sample] = False
        sample_unphaseable[sample] = ''
        aa_haplotype[sample] = ''
        nucleotide_haplotype[sample] = ''
        num_missing[sample] = 0
        num_called[sample] = 0
        swap_phasing[sample] = False
        first_PGT[sample] = ''

    for i in np.arange(len(starts)):
        start = starts[i]
        end = ends[i]
        exon_offset = exon_offsets[i]
        if verbose:
            print(f"{chrom}:{starts[i]}-{ends[i]}")
        for record in vcf_reader.fetch(chrom, start-1, end): # Find variants in relevant portion of gene
            if ( record.POS >= start) and not record.POS in positions_to_ignore: # Have to restrict to position that are at or after start, to ensure indels in previous positions don't mess things up. Note this is a change from previous behaviour
                if not use_filter or record.FILTER==[]: # Only use PASS variants, unless use_filter is False
                    if verbose:
                        print("\n", record, record.FILTER)
                    for sample in samples:
                        if show_genotypes:
                            print(record.genotype(sample))
                        GT = record.genotype(sample)['GT']
                        AD = record.genotype(sample)['AD']
                        POS = record.POS - start + exon_offset
                        if GT == './.' or GT == '.|.': # Note phased empty genotypes (.|.) have appeared in Pf7. These weren't in Pf6!
                            if not record.POS in positions_with_missingness:
                                num_missing[sample] += 1
                        else:
                            num_called[sample] += 1
                            if GT != '0/0' and GT != '0|0': # Only interested in non-ref genotypes. Hom ref genotypes don't change sequence so can be ignored
                                alleles = [record.REF] + record.ALT
#                                 first_allele_int = int(GT[0])
#                                 second_allele_int = int(GT[2])
                                # Find lengths of alleles (to determine which are indels)
                                REF_len = len(record.REF)
                                first_allele_ALT_len = len(alleles[int(GT[0])])
                                second_allele_ALT_len = len(alleles[int(GT[2])])
                                if (REF_len==first_allele_ALT_len and REF_len==second_allele_ALT_len) or include_indels or record.POS in positions_to_allow:
                                    if (GT[0] != GT[2]) and (alleles[int(GT[0])] != '*') and (alleles[int(GT[2])] != '*'): # heterozygous call at non-spanning deletion
                                        if 'PGT' in record.genotype(sample).data._fields:
                                            is_unphased = (record.genotype(sample)['PGT'] is None)
                                        else:
                                            is_unphased = True
                                        if non_phased_het_seen[sample]:
                                            if verbose:
                                                print("Sample %s unphased het followed by het" % sample)
                                            sample_unphaseable[sample] = '*'
            #                                 genotype[sample] = 'X'
                                        if is_unphased:
                                            non_phased_het_seen[sample] = True
                                        else:
#                                             if 'PID' in record.genotype(sample).data._fields: # Note this won't be the case in het spanning deletions
                                            if PID[sample] == '':
                                                PID[sample] = record.genotype(sample)['PID']
                                                first_PGT[sample] = record.genotype(sample)['PGT']
                                                if AD[int(GT[2])] > AD[int(GT[0])]:
                                                    swap_phasing[sample] = True
                                            else:
                                                if record.genotype(sample)['PID'] != PID[sample]:
                                                    if verbose:
                                                        print("Sample %s two PIDs" % sample)
                                                    sample_unphaseable[sample] = '*'
            #                                         genotype[sample] = '?'
#                                             if record.genotype(sample)['PGT'] == '1|0':
#                                                 GT = GT[2] + '/' + GT[0]
                                        if verbose:
                                            print(f"AD={AD}, GT={GT}")
                                        # Here we use phased genotype (PGT) if in same phasing group as first het
                                        if (
                                            'PID' in record.genotype(sample).data._fields
                                            and ( PID[sample] == record.genotype(sample)['PID'] )
                                        ): # In phasing group with first het call
                                            if verbose:
                                                print(f"PID: {PID[sample]}, GT: {GT}, PGT: {record.genotype(sample)['PGT']}")
                                            if first_PGT[sample] == record.genotype(sample)['PGT']:
                                                if swap_phasing[sample]:
                                                    GT = GT[2] + '/' + GT[0]
                                            else:
                                                if not swap_phasing[sample]:
                                                    GT = GT[2] + '/' + GT[0]
#                                             GT = record.genotype(sample)['PGT']
                                        # Else we ensure we are taking the majority haplotype as the first allele
                                        elif AD[int(GT[2])] > AD[int(GT[0])]:
                                            GT = GT[2] + '/' + GT[0]
                                            if verbose:
                                                print(f"AD={AD}, GT={GT}")

                                for i, sample_offset in enumerate(sample_offsets[sample]):
                                    alleles = [record.REF] + record.ALT
                                    GTint = int(GT[i*2])
                                    REF_len = len(record.REF)
                                    ALT_len = len(alleles[GTint])
                                    if(verbose):
                                        print(f"before: i={i}, offset={sample_offset}, alleles, GT={GTint}, REF={REF_len}, ALT={ALT_len}, {sample_sequences[sample][i]}")
                                    if GTint != 0 and alleles[GTint] != '*' and (REF_len==ALT_len or include_indels or record.POS in positions_to_allow):
                                        sample_sequences[sample][i][POS+sample_offsets[sample][i]:(POS+sample_offsets[sample][i]+REF_len)] = alleles[GTint]
                                        sample_offsets[sample][i] = sample_offset + (len(alleles[GTint]) - len(record.REF))
                                        if(verbose):
                                            print(f"after : i={i}, offset={sample_offset}, alleles, GT={GTint}, REF={REF_len}, ALT={ALT_len}, {sample_sequences[sample][i]}")

    for sample in samples:
        if(show_genotypes):
            print(sample_sequences[sample])
        ###########################
        # populate ns_changes
        ###########################
        if strand == '+':
            aa_0 = (
                str(sample_sequences[sample][0].toseq().translate(to_stop=to_stop))
                if len(sample_sequences[sample][0]) % 3 == 0
                else '!'
            )
            aa_1 = (
                str(sample_sequences[sample][1].toseq().translate(to_stop=to_stop))
                if len(sample_sequences[sample][1]) % 3 == 0
                else '!'
            )
        else:
            aa_0 = (
                str(sample_sequences[sample][0].toseq().reverse_complement().translate(to_stop=to_stop))
                if len(sample_sequences[sample][0]) % 3 == 0
                else '!'
            )
            aa_1 = (
                str(sample_sequences[sample][1].toseq().reverse_complement().translate(to_stop=to_stop))
                if len(sample_sequences[sample][1]) % 3 == 0
                else '!'
            )
#         if aa_0 != '!' and aa_1 != '!' # This was what was used in Pf6
        if aa_0 != '!' and aa_1 != '!' and (len(aa_0) == len(ref_aa_haplotype)) and (len(aa_1) == len(ref_aa_haplotype)): # In Pf7, we have some in-frame indels, hence we also need to check for these, and give "!" if indels exist
            if aa_0 == aa_1:
                ns_changes_list = []
                for i in range(len(ref_aa_haplotype)):
                    if aa_0[i] != ref_aa_haplotype[i]:
                        ns_changes_list.append("%s%d%s" % (ref_aa_haplotype[i], i+first_aa, aa_0[i]))
                ns_changes[sample] = "/".join(ns_changes_list)
            else:
                ns_changes_list_0 = []
                for i in range(len(ref_aa_haplotype)):
                    if aa_0[i] != ref_aa_haplotype[i]:
                        ns_changes_list_0.append("%s%d%s" % (ref_aa_haplotype[i], i+first_aa, aa_0[i]))
                ns_changes_0 = "/".join(ns_changes_list_0)
                ns_changes_list_1 = []
                for i in range(len(ref_aa_haplotype)):
#                     print(len(ref_aa_haplotype), len(aa_1))
                    if aa_1[i] != ref_aa_haplotype[i]:
                        ns_changes_list_1.append("%s%d%s" % (ref_aa_haplotype[i], i+first_aa, aa_1[i]))
                ns_changes_1 = "/".join(ns_changes_list_1)

                if ns_changes_0 == '':
                    ns_changes[sample] = ns_changes_1.lower()
                elif ns_changes_1 == '':
                    ns_changes[sample] = ns_changes_0.lower()
                else:
                    ns_changes[sample] = (",".join([ns_changes_0, ns_changes_1])).lower()
        else:
            if verbose:
                print(aa_0, aa_1, ref_aa_haplotype)
            ns_changes[sample] = '!'

        ################################################
        # populate nucleotide_haplotype and aa_haplotype
        ################################################
        if str(sample_sequences[sample][0]).upper() == str(sample_sequences[sample][1]).upper():
            nucleotide_haplotype[sample] = str(sample_sequences[sample][0].toseq())
            if strand == '+':
                aa_haplotype[sample] = ( str(sample_sequences[sample][0].toseq().translate(to_stop=to_stop))
                                    if len(sample_sequences[sample][0]) % 3 == 0
                                    else '!' )
            else:
                aa_haplotype[sample] = ( str(sample_sequences[sample][0].toseq().reverse_complement().translate(to_stop=to_stop))
                                    if len(sample_sequences[sample][0]) % 3 == 0
                                    else '!' )
        else:
            nucleotide_haplotype[sample] = "%s,%s" % ( str(sample_sequences[sample][0].toseq()), str(sample_sequences[sample][1].toseq()) )
            if strand == '+':
                aa_haplotype[sample] = "%s,%s" % (
                    ( str(sample_sequences[sample][0].toseq().translate(to_stop=to_stop))
                     if len(sample_sequences[sample][0]) % 3 == 0
                     else '!' ),
                    ( str(sample_sequences[sample][1].toseq().translate(to_stop=to_stop))
                     if len(sample_sequences[sample][1]) % 3 == 0
                     else '!' )
                )
            else:
                aa_haplotype[sample] = "%s,%s" % (
                    ( str(sample_sequences[sample][0].toseq().reverse_complement().translate(to_stop=to_stop))
                     if len(sample_sequences[sample][0]) % 3 == 0
                     else '!' ),
                    ( str(sample_sequences[sample][1].toseq().reverse_complement().translate(to_stop=to_stop))
                     if len(sample_sequences[sample][1]) % 3 == 0
                     else '!' )
                )
    
        # Add asterisk to end if sample is unphaseable
        ns_changes[sample] += sample_unphaseable[sample]
        aa_haplotype[sample] += sample_unphaseable[sample]
        nucleotide_haplotype[sample] += sample_unphaseable[sample]

        # Set ns_changes output to missing ('-') if no NS changes and at least one missing genotype call in the region
        if (ns_changes[sample] == '' or ns_changes[sample] == '*') and num_missing[sample] > 0:
            ns_changes[sample] = '-'
                
        # Set sequence output to missing ('-') if any missing genotype call in the gene
        if num_missing[sample] > 0:
            aa_haplotype[sample] = '-'
            nucleotide_haplotype[sample] = '-'
    
    df_all_haplotypes = pd.DataFrame(
        {
            'aa_haplotype': pd.Series(aa_haplotype),
            'nucleotide_haplotype': pd.Series(nucleotide_haplotype),
            'ns_changes': pd.Series(ns_changes),
        }
    )
        
        
#     return(df_haplotypes, ref_aa_haplotype, ref_nucleotide_haplotype)
    return(
        {
            'df_all_haplotypes':df_all_haplotypes,
            'ref_aa_haplotype':ref_aa_haplotype,
            'ref_nucleotide_haplotype':ref_nucleotide_haplotype,
        }
    )


In [8]:
def prepare_plot_data(
    gene_id='PF3D7_0709000',
    haplotype_calls=None,
    samples_fn=samples_fn,
    expected_aa_len=None,
#     populations = ['SA', 'AF-W', 'AF-C', 'AF-NE', 'AF-E', 'AS-S-E', 'AS-S-FE', 'AS-SE-W', 'AS-SE-E', 'OC-NG', ],
    populations = ['AF-W', 'AF-C', 'AF-NE', 'AF-E', ],
    samples_to_show = {
        'PG0563-C': '7G8',
        'PG0564-C': 'GB4',
        'PG0565-C': 'HB3',
        'PG0566-C': 'Dd2',
        'PG0567-C': '3D7',
        'PG0568-C': 'IT',
    },
    sample_to_use_as_background = 'PG0566-C', # Dd2
    verbose=True,
    **kwargs
):
#     print(gene_id)
    if gene_id is not None:
        cds_results = determine_cds(gene_id)
#     print(cds_results['gene_name'])
    
        if haplotype_calls is None:
            haplotype_calls = call_haplotype(
                chrom=cds_results['chrom'],
                starts=cds_results['starts'],
                ends=cds_results['ends'],
                strand=cds_results['strand'],
                **kwargs
            )
    else:
        haplotype_calls = call_haplotype(
            **kwargs
        )
        
    
    df_samples = pd.read_csv(samples_fn, sep='\t', low_memory=False, index_col=0)
    if verbose:
        print(f"Read in {df_samples.shape[0]:,} samples, of which {np.count_nonzero(df_samples['QC pass']):,} passed QC")

    df_join = df_samples.join(haplotype_calls['df_all_haplotypes'])
    
    if expected_aa_len is None:
        expected_aa_len = len(haplotype_calls['ref_aa_haplotype'])
    
    df_clonal = df_join.loc[
        (
            ( df_join['QC pass'] == True )
            & ( df_join['aa_haplotype'].astype(str).apply(len) == expected_aa_len )
        ) | ( df_join['Study'] == '1153-PF-Pf3KLAB-KWIATKOWSKI' ),
        [
            'aa_haplotype',
            'nucleotide_haplotype',
            'ns_changes',
            'Population',
        ]
    ]
    
    if df_clonal.shape[0] == 0:
        most_common_length = df_join['aa_haplotype'].astype(str).apply(len).value_counts().index[0]
        raise Exception(f"There were no samples with AA haplotype length = {expected_aa_len}. Most common length was {most_common_length}")
    
    df_clonal['ns_changes'].fillna('', inplace=True)
    df_clonal['number_of_mutations'] = df_clonal['ns_changes'].apply(lambda x: len(x.split('/')))
    df_clonal.loc[df_clonal['ns_changes'] == '', 'number_of_mutations'] = 0
    if verbose:
        print(f"Found {df_clonal.shape[0]:,} clonal haplotypes in QC pass samples")

    grouping_columns = [
        'number_of_mutations', 'ns_changes',
    #     'haplotype_name',
    ]
    ascending=[True]*len(grouping_columns)
    ascending.append(False)

    def n_agg(x):
        names = collections.OrderedDict()
        for population in populations:
            names[population] = np.count_nonzero( (x['Population'] == population) )
        return pd.Series(names)

    df_haplotypes = df_clonal.groupby(grouping_columns).apply(n_agg)
    df_haplotypes['Total'] = df_haplotypes.sum(axis=1)
    df_haplotypes.reset_index(inplace=True)
    df_haplotypes.sort_values(grouping_columns + ['Total'], ascending=ascending, inplace=True)
    df_haplotypes['ns_changes_list'] = df_haplotypes['ns_changes'].apply(lambda x: x.split('/'))
    df_haplotypes.sort_values('Total', ascending=False, inplace=True)
    # Determine which samples to show have which haplotypes
    df_haplotypes['sample_names'] = ''
    for sample in samples_to_show.keys():
        ns_changes = df_clonal.loc[sample, 'ns_changes']
        if df_haplotypes.loc[df_haplotypes['ns_changes']==ns_changes, 'sample_names'].values == '':
            df_haplotypes.loc[df_haplotypes['ns_changes']==ns_changes, 'sample_names'] = samples_to_show[sample]
        else:
            df_haplotypes.loc[df_haplotypes['ns_changes']==ns_changes, 'sample_names'] = (
                df_haplotypes.loc[df_haplotypes['ns_changes']==ns_changes, 'sample_names'] + '\n' + samples_to_show[sample]
            )
    if verbose:
        print(f"Found {df_haplotypes.shape[0]:,} unique haplotypes in QC pass samples")
        
    # Determine background mutations
    if sample_to_use_as_background in df_clonal.index:
        background_ns_changes = df_clonal.loc[sample_to_use_as_background, 'ns_changes']
        
    if gene_id is not None:
        gene_name = cds_results['gene_name']
    else:
        gene_name = None
    
    
    return(
        {
            'df_haplotypes':df_haplotypes,
            'df_clonal':df_clonal,
            'df_join':df_join,
            'background_ns_changes':background_ns_changes,
            'haplotype_calls':haplotype_calls,
            'gene_name':gene_name,
            'df_samples':df_samples,
        }
    )

    

In [9]:
# plot for DRF

def drug_plot(drug, drug_name, populations, label): 
    
    from matplotlib.ticker import MaxNLocator

    plt.rcParams['axes.labelsize'] = 4
    plt.rcParams['axes.titlesize'] = 5
    
    plt.rcParams['xtick.labelsize'] = 3
    plt.rcParams['ytick.labelsize'] = 4
    
    plt.rcParams['ytick.alignment'] = 'bottom'
    
    plt.rcParams['figure.dpi'] = 1000
    
    
    xlims = [(1982, 1986),
            (1994,1998),
            (2000,2019)]
    
    fig, axs = plt.subplots(1, 3, gridspec_kw={'width_ratios': [1,1,5]}, figsize=((16/5),9), sharey=True)
                
    ax, ax1, ax2 = axs
    fig.subplots_adjust(hspace=0.05)
    
    first_label_ax2 = True 
    
    for i,ax in enumerate(axs):

        for pop in reversed(populations):
            
            country = drug[drug['Population'] == pop]['Country']
            _, idx = np.unique(country, return_index=True)
            country = country.reset_index(drop=True)

            for c in country[np.sort(idx)]:

                data = drug[(drug['Population'] == pop) & (drug['Country'] == c)]

                for index, row in data.iterrows():
                    
                    if  (row[drug_name + ' frequency'] == 1):
                        ax.plot('Year', 'Label', '-o', data=row, color = 'black',markeredgewidth=0.3, markersize=4)
                        # plot x marker for one DR frequency 
                        ax.plot('Year', 'Label', '-x', data=row, color = 'white',markeredgewidth=0.3, markersize=4)
                        
                    elif  (row[drug_name + ' frequency'] == 0):
                        ax.plot('Year', 'Label', '-o', data=row, markeredgecolor= 'black',  markerfacecolor='None',  markeredgewidth=0.3, markersize=4)
                        # plot x marker for zero DR frequency 
                        ax.plot('Year', 'Label', '-x', data=row, color = 'black', markeredgewidth=0.3, markersize=4)
                    
    
                    elif np.isnan(row[drug_name + ' frequency']) :

                        # add the  black edge colour for each point
                        ax.plot('Year', 'Label', '-o', data=row, markeredgecolor= 'white', alpha=0,  markerfacecolor='None', markersize=4)

                    else:
                        # add the  black edge colour for each point
                        ax.plot('Year', 'Label', '-o', data=row, markeredgecolor= 'black',  markerfacecolor='None',  markeredgewidth=0.3, markersize=4)
                        # plot DR frequency with alpha value
                        ax.plot('Year', 'Label', '-o', data=row, color = 'black', alpha=(row[drug_name +' frequency']),  markeredgewidth=0.3, markersize=4)

                    x1,x2 = xlims[i]
                    ax.set_xlim(x1,x2)
                    
                    ax.grid(axis='y', color = 'grey', linewidth = '0.25')
                        
                    if i == 2:
                        ax.spines['left'].set_visible(False)
                        ax.tick_params(axis='y', colors='None')
                        #ax.set_axisbelow(True)
                       
                        
                    elif i == 1:
                        ax.spines['right'].set_visible(False)
                        ax.spines['left'].set_visible(False)
                        #ax.set_axisbelow(True)
                        ax.tick_params(axis='y', colors='None')
                        

                        
                    
                    elif i == 0:

                        ax.spines['right'].set_visible(False)
                        ax.tick_params(axis='y', colors='None')
                        
                    ax.xaxis.set_minor_locator(plt.MaxNLocator(x2-x1))
                    
                            

    ax.set_title(drug_name, loc='left')
    
    #x axis years as integers           
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    
    #set yaxis with population colours
    labels = []
#     labels += [population_colours['OC-NG']]*label['OC-NG']
#     labels += [population_colours['AS-SE-E']]*label['AS-SE-E']
#     labels += [population_colours['AS-SE-W']]*label['AS-SE-W']

#     labels += [population_colours['AS-S-FE']]*label['AS-S-FE']
#     labels += [population_colours['AS-S-E']]*label['AS-S-E']

    labels += [population_colours['AF-E']]*label['AF-E']
    labels += [population_colours['AF-NE']]*label['AF-NE']
    labels += [population_colours['AF-C']]*label['AF-C']
    labels += [population_colours['AF-W']]*label['AF-W']
#     labels += [population_colours['SA']]*label['SA']


    ax.tick_params(axis='y',direction='out', pad=58, reset = True)
   

    [(t.set_color(labels[n]),t.set_horizontalalignment('left'), t.set_fontsize(4)) for n,t in enumerate(ax.yaxis.get_ticklabels())]

    ax.tick_params(axis='y', which='both', length=0, bottom = True)
    ax.grid(axis='y', color = 'grey', linewidth = '0.25')
                        
                        
    plt.subplots_adjust(wspace=0.15)
    
    return(fig)


In [10]:
def locations_agg(x, ns_changes='F313S/Q454E'):
    names = collections.OrderedDict()
    names['n'] = np.count_nonzero(x['ns_changes_homozygous'])
    if names['n'] == 0:
        names[f'{ns_changes} frequency'] = np.nan
    else:
        names[f'{ns_changes} frequency'] = np.count_nonzero(
            ( x['ns_changes'] == ns_changes)
        ) / names['n']
    
    return pd.Series(names)



In [11]:
def haplotype_abacus_plot(
    ns_changes = 'F313S/Q454E',
    min_samples = 25,
    ppd_results = None,
    df_samples = None,
#     populations = ['SA', 'AF-W', 'AF-C', 'AF-NE', 'AF-E', 'AS-S-E', 'AS-S-FE', 'AS-SE-W', 'AS-SE-E', 'OC-NG'],
    populations = ['AF-W', 'AF-C', 'AF-NE', 'AF-E', ],
    png_fn=None,
    dpi=240,
    pdf_fn=None,
):
    df_samples_with_ns_changes = df_samples.join(ppd_results['haplotype_calls']['df_all_haplotypes']['ns_changes'])
    df_samples_with_ns_changes = df_samples_with_ns_changes.loc[df_samples_with_ns_changes['QC pass']]
    df_samples_with_ns_changes.loc[df_samples_with_ns_changes['Country'] == 'Democratic Republic of the Congo', ['Country']] = 'DRC'

    df_samples_with_ns_changes['ns_changes_homozygous'] = ( df_samples_with_ns_changes['ns_changes'] == df_samples_with_ns_changes['ns_changes'].str.upper() )
    
    df_frequencies = (
        df_samples_with_ns_changes
        .groupby(['Population', 'Year', 'Country', 'Admin level 1'])
        .apply(lambda x: locations_agg(x, ns_changes))
        .reset_index()
        .set_index(['Population', 'Country', 'Admin level 1'])
        .loc[locations]
        .reset_index()
    )
    df_frequencies = df_frequencies.loc[df_frequencies['n'] >= min_samples]
    df_frequencies['Label'] = df_frequencies['Country'] + ', ' + df_frequencies['Admin level 1']
    
    label = {}
    for p in populations: 
        arr = np.unique(df_frequencies[df_frequencies['Population'] == p]['Admin level 1'], return_counts= True)
        label.update({p: len(arr[1])})

    fig = drug_plot(df_frequencies, ns_changes, populations, label)

    if png_fn is not None:
        fig.savefig(png_fn, dpi=dpi)
    if pdf_fn is not None:
        fig.savefig(pdf_fn)

#     return(fig)


In [12]:
def haplotypes_plot(
    gene_id=None,
    prepare_plot_data_results=None,
    gene_name='',
    min_samples=5,
    figsize=(12, 12),
    max_ylim=5000,
    sample_counts_plot_log='y',
    cumulative_proportions_text=None,
    background_ns_changes=None,
    create_abacus_plots=True,
    min_samples_for_abacus_plots=25,
    png_fn=None,
    dpi=240,
    pdf_fn=None,
#     background_ns_changes='M74I/N75E/K76T/A220S/Q271E/N326S/I356T/R371I',
    **kwargs
):
    if cumulative_proportions_text is None:
        if sample_counts_plot_log == 'y':
            cumulative_proportions_text = 'below'
        else:
            cumulative_proportions_text = 'above'
    
    return_values = 'fig'
    
    if prepare_plot_data_results is None:
        return_values = 'all'
        prepare_plot_data_results = prepare_plot_data(gene_id=gene_id, **kwargs)
    df_haplotypes = prepare_plot_data_results['df_haplotypes']
    if gene_name == '':
        gene_name = prepare_plot_data_results['gene_name']

    if background_ns_changes is None:
        background_ns_changes = prepare_plot_data_results['background_ns_changes']

    fig, axs = plt.subplots(3, 1, figsize=figsize, gridspec_kw={'height_ratios': [1, 0.5, 2]})
    
    total_samples = df_haplotypes['Total'].sum()
    df_haplotypes['cum_proportion'] = df_haplotypes['Total'].cumsum() / total_samples
    
    df_haplotypes_set = df_haplotypes.loc[df_haplotypes['Total'] >= min_samples]

    mutations = pd.Series(np.unique(np.concatenate(df_haplotypes_set['ns_changes_list'].values)))
    mutations = mutations.loc[mutations != '']
    mutations_np_ref = mutations.apply(lambda x: x[1:])
    aa = mutations_np_ref.apply(lambda x: int(x[:-1]))
    df_mutations_set = pd.DataFrame(
        {
            'mutation': mutations.values,
            'aa': aa.values,
        }
    ).sort_values('aa').reset_index(drop=True).reset_index().set_index('mutation')

    # Breakdown by number of samples (log scale)
    axs[0].bar(df_haplotypes_set['ns_changes'], df_haplotypes_set['Total'], color='black', log=sample_counts_plot_log)
    axs[0].set_xticklabels([])
    if sample_counts_plot_log == 'y':
        axs[0].set_ylim(0.5, max_ylim)
    if sample_counts_plot_log=='y':
        yticklabel_values = [0, 0] + [10**x for x in np.arange(int(np.ceil(np.log10(max_ylim + 1))))]
        axs[0].set_yticklabels(yticklabel_values)
        axs[0].set_ylabel('Number of samples (log scale)', fontsize=10)
    else:
        axs[0].set_ylabel('Number of samples', fontsize=10)
    if cumulative_proportions_text=='below':
        for i in np.arange(df_haplotypes_set.shape[0]):
            axs[0].text(
                i,
                df_haplotypes_set.iloc[i]['Total'],
                f"{df_haplotypes_set.iloc[i]['cum_proportion']:.1%}",
                rotation=90,
                ha='center',
                va='top',
                color='white'
            )
    if cumulative_proportions_text=='above':
        for i in np.arange(df_haplotypes_set.shape[0]):
            axs[0].text(
                i,
                df_haplotypes_set.iloc[i]['Total'],
                f"{df_haplotypes_set.iloc[i]['cum_proportion']:.1%}",
                rotation=90,
                ha='center',
                va='bottom',
                color='black'
            )

    # Breakdown by population
    for column_num in np.arange(2, df_haplotypes_set.shape[1]-4):
        pop = df_haplotypes_set.columns[column_num]
#         print(pop)

        label=pop
        if column_num == 2:
            axs[1].bar(
                df_haplotypes_set['ns_changes'],
                df_haplotypes_set.iloc[:, column_num] / df_haplotypes_set.loc[:, 'Total'],
                color=population_colours[pop],
                label=label
            )
        else:
            axs[1].bar(
                df_haplotypes_set['ns_changes'],
                df_haplotypes_set.iloc[:, column_num] / df_haplotypes_set.loc[:, 'Total'],
                color=population_colours[pop],
                label=label, 
                bottom=df_haplotypes_set.iloc[:, 2:column_num].sum(axis=1) / df_haplotypes_set.loc[:, 'Total'],
            )

    axs[1].set_xticklabels(df_haplotypes_set['sample_names'])
    
    l = axs[1].legend(loc="lower left",  ncol = len(population_colours), bbox_to_anchor=(0.1, 1.0), frameon=False)

    # Mutations in each haplotype
    i = 0
    if ( not '' in background_ns_changes ) and background_ns_changes in df_haplotypes_set['ns_changes'].values:
        background_mutation_indices = df_mutations_set.loc[
            df_haplotypes_set.loc[
                df_haplotypes_set['ns_changes'] == background_ns_changes,
                'ns_changes_list'
            ].values[0],
            'index'
        ].values
    else:
        background_mutation_indices = []

    for ix, row in df_haplotypes_set.iterrows():
        if row['ns_changes_list'][0] != '':
            indexes = df_mutations_set.loc[row['ns_changes_list'], 'index'].values
            axs[2].plot([i] * len(indexes), indexes, '-', linewidth=3, color='grey')
            background_mutations = np.intersect1d(indexes, background_mutation_indices)
            other_mutations = np.setdiff1d(indexes, background_mutation_indices)
            axs[2].scatter([i] * len(background_mutations), background_mutations, s=100, color='grey', zorder=3)
            axs[2].scatter([i] * len(other_mutations), other_mutations, s=100, color='black', zorder=3)
        i = i + 1
    axs[2].set_xlim(axs[0].get_xlim())
    axs[2].set_xticks([])
    axs[2].set_yticks(np.arange(df_mutations_set.shape[0]))
    axs[2].set_yticklabels(df_mutations_set.index)
    axs[2].grid(axis = 'y')
    axs[2].spines['top'].set_visible(False)
    axs[2].spines['right'].set_visible(False)
    axs[2].spines['bottom'].set_visible(False)
    axs[2].spines['left'].set_visible(False)

    axs[1].set_ylabel('Proportion of samples', fontsize=10)
#     axs[2].set_xlabel('All $crt$ haplotypes with Dd2 background', fontsize=10)
    axs[2].set_xlabel('Haplotypes', fontsize=10)

    axs[0].set_title(gene_name)
    
    fig.tight_layout()
    
    if png_fn is not None:
        fig.savefig(png_fn, dpi=dpi)
    if pdf_fn is not None:
        fig.savefig(pdf_fn)
        
    
    if create_abacus_plots:
        for ns_changes in df_haplotypes_set['ns_changes']:
#             print(ns_changes)
            if png_fn is None:
                abacus_png_fn = None
            else:
                abacus_png_fn = png_fn.replace(".png", f"_{ns_changes.replace('/', '_')}.png")
            if pdf_fn is None:
                abacus_pdf_fn = None
            else:
                abacus_pdf_fn = pdf_fn.replace(".pdf", f"_{ns_changes.replace('/', '_')}.pdf")
            haplotype_abacus_plot(
                ns_changes = ns_changes,
                min_samples = min_samples_for_abacus_plots,
                ppd_results = prepare_plot_data_results,
                df_samples = prepare_plot_data_results['df_samples'],
                png_fn = abacus_png_fn,
                pdf_fn = abacus_pdf_fn,
            )
    
    if return_values == 'all':
        return(
            {
                'fig':fig,
                'df_haplotypes':prepare_plot_data_results['df_haplotypes'],
                'df_clonal':prepare_plot_data_results['df_clonal'],
                'df_join':prepare_plot_data_results['df_join'],
                'background_ns_changes':background_ns_changes,
                'haplotype_calls':prepare_plot_data_results['haplotype_calls'],
            }
        )
    elif return_values == 'fig':
        return(fig)


# Create plots

In [13]:
genes_of_interest = collections.OrderedDict()

genes_of_interest['PF3D7_0417200'] = 'DHFR'
genes_of_interest['PF3D7_0523000'] = 'MDR1'
genes_of_interest['PF3D7_0709000'] = 'CRT'
genes_of_interest['PF3D7_0810800'] = 'DHPS'
genes_of_interest['PF3D7_1224000'] = 'GCH1'

In [14]:
%%time
%matplotlib
# %matplotlib inline
results=collections.OrderedDict()
for gene_id in genes_of_interest:
    print(gene_id, genes_of_interest[gene_id])
    !mkdir -p plots/png/{gene_id}
    png_fn = f"plots/png/{gene_id}/{gene_id}.png"
    !mkdir -p plots/pdf/{gene_id}
    pdf_fn = f"plots/pdf/{gene_id}/{gene_id}.pdf"
    results[genes_of_interest[gene_id]] = haplotypes_plot(gene_id=gene_id, sample_counts_plot_log='', png_fn=png_fn, pdf_fn=pdf_fn)
    print()

Using matplotlib backend: agg
PF3D7_0417200 DHFR
Read in 20,864 samples, of which 16,203 passed QC
Found 13,781 clonal haplotypes in QC pass samples
Found 30 unique haplotypes in QC pass samples
7
AF-W
AF-C
AF-NE
AF-E

PF3D7_0523000 MDR1
Read in 20,864 samples, of which 16,203 passed QC
Found 10,239 clonal haplotypes in QC pass samples
Found 323 unique haplotypes in QC pass samples
7
AF-W
AF-C
AF-NE
AF-E

PF3D7_0709000 CRT
Read in 20,864 samples, of which 16,203 passed QC
Found 12,405 clonal haplotypes in QC pass samples
Found 101 unique haplotypes in QC pass samples
7
AF-W
AF-C
AF-NE
AF-E

PF3D7_0810800 DHPS
Read in 20,864 samples, of which 16,203 passed QC
Found 12,533 clonal haplotypes in QC pass samples
Found 78 unique haplotypes in QC pass samples
7
AF-W
AF-C
AF-NE
AF-E

PF3D7_1224000 GCH1
Read in 20,864 samples, of which 16,203 passed QC
Found 14,886 clonal haplotypes in QC pass samples
Found 97 unique haplotypes in QC pass samples
7
AF-W
AF-C
AF-NE
AF-E

CPU times: user 30min 32

In [15]:
!date

Wed Dec 21 20:05:54 GMT 2022


# Conclusions

Here we have created haplotype and "abacus" plots for the five genes of interest. Some brief highlights from a quick look at the output plots include:
- There are two haplotypes containing DHPS A581G
    - K540E/A581G is seen mostly in East Africa, though very rarely in Kenya
    - I431V/S436A/A581G/A613S is seen exclusively in west and central Africa, and is most common in Benin, Nigeria and Cameroon
- We see two different double-mutants of DHFR, C59R/S108N and N51I/S108N, with the latter being relatively rare in west Africa, but almost at fixation in Sudan. 
- We don't see more than one amino acid mutation in any haplotype of GCH1 seen in at least 5 samples.

